In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
file1 = '../ArchConfAbstracts/ECSA2007.txt'

In [ ]:
import re
non_decimal = re.compile('[^a-zA-Z]')

# Working with a Single file
- using all labels
- note issues in the columns, not all entries are the same 
- - must change the structure


In [ ]:
import singleFile as parser1
reload(parser1)

In [ ]:
testdict = parser1.parseFile(file1)

In [ ]:
for k,v in testdict.iteritems():
    if k == 0:
        pass
    elif k == 1:
        print k
        f = pd.DataFrame(v, columns = ['Labels','Values'])
        f = f.set_index('Labels').T
        columns = f.columns
    elif k == 2:
        g = pd.DataFrame(v, columns = ['Labels','Values'])
        g = g.set_index('Labels').T
print columns

In [ ]:
testdict[1]

In [ ]:
for k,v in testdict.iteritems():
    if k == 0:
        pass
    elif k == 1:
        print k
        f = pd.DataFrame(v, columns = ['Labels','Values'])
        f = f.set_index('Labels').T
        columns = f.columns
    else:
        try:
            g = pd.DataFrame(v, columns = ['Labels','Values'])
            g = g.set_index('Labels').T
            f.append(g, ignore_index = True)
        except:
            print k

## As a dataframe!

In [ ]:
testdf = parser1.parseEntrytoDF(file1)
testdf.head()

In [ ]:
import re
regXsuperscript = re.compile('/\p{No}/gu/')

In [ ]:
def testme(filethis):
    f = open(filethis, "r")
    i = 1
    abstracts = {}
    entry = []
    nonlabels = ['authors', 'copyright', 'UNKNOWN']
    for line in f:
        if line[0].isdigit():
            abstracts[i-1] = entry
            entry = []
            entry.append((str(i), regXsuperscript.sub(' ', line).strip()))
            i = i + 1
        else:
            try:
                #non_decimal = re.compile('[\W_]+')
                label, text = line.split(':', 1)
                text = regXsuperscript.sub(' ', text)
                label = regXsuperscript.sub(' ', label)
                entry.append((label.strip(), text.strip()))
            except:
                if line is None:
                    pass
                else:
                    text = regXsuperscript.sub(' ', line)
                    label = 'reparse'
                    entry.append((label, text.strip()))
                    
                
    f.close()
    return abstracts

In [ ]:
f = testme(file1)

In [ ]:
f

##### appending dataframes together, 
- Note the descrepanies - -> look into the issues (Decide to change the structure)

In [ ]:
x  = testdf.append(testdf)

In [ ]:
testdf.describe()

In [ ]:
testdf['IPC Code']

In [184]:
import parseFiles as parser2
parser2 = reload(parser2)

In [185]:
retest2 = parser2.parseEntrytoDF(file1)

In [189]:
retest2['Authors'].iloc[0]

"['Falessi, D', ' Kruchten, P', ' Cantone, G']"

In [187]:
retest2['Authors'].iloc[1]

"['Jerad, C', ' Barkaoui, K', ' Grissa Touzi, A']"

In [188]:
retest2.iloc[1]['Authors']

"['Jerad, C', ' Barkaoui, K', ' Grissa Touzi, A']"

In [112]:
def reformatcode(classcodes):
    try:
        codes = [x.split(' ',1)[1] for x in classcodes.split('   ')]
        return codes
    except:
        return classcodes
        
def reformatTerms(terms):
    from itertools import chain
    try:
        terms = [entry.split(' - ') for entry in terms]
        return list(chain(*terms))
    except:
        return 'ERROR' + str(terms)
    
    

In [113]:
retest['Classification Code'] = retest['Classification Code'].apply(reformatcode)

In [114]:
print retest.iloc[0]['Classification Code']
print retest.iloc[22]['Classification Code']
print retest.iloc[0]['terms']
print retest.iloc[22]['terms']

['Software engineering techniques', 'Software management']
nan
['software architecture - software quality', 'empirical software engineering - software architecture - software economics - software quality']
[]


In [115]:
retest['terms1'] = retest.terms.apply(reformatTerms)
print retest.iloc[0]['terms1']
print retest.iloc[22]['terms1']

['software architecture', 'software quality', 'empirical software engineering', 'software architecture', 'software economics', 'software quality']
[]


In [105]:
retest['newterm'] = retest['terms1'] + retest['Classification Code']

In [106]:
print retest.iloc[0]['newterm']
print retest.iloc[22]['newterm']

['software architecture', 'software quality', 'empirical software engineering', 'software architecture', 'software economics', 'software quality', 'Software engineering techniques', 'Software management']
nan


In [ ]:
def parseTerms(tableDF, keyword = 'terms'):
    ''' Parse the TOTALABSTRACTS table dataframe terms column.
        Create a single set of all terms
        
        : param tableDF : a Pandas DataFrame (ie TOTALABSTRACTS table as DF)
        : param keyword : column name to be recast and set of terms found from
        
        : output : a master set of terms set as a list (no duplicates)
    '''
    
    df = tableDF
    
    #recast - since made a string during the entry into the sqlite db.
    df[keyword] = df[keyword].apply(lambda kw : ','.join(kw.split('-')))
    
    #strip the str from prior manipulation
    terms = df[keyword].apply(lambda x: set([ e.strip(' \'') for e in x.strip('[]\'').split(',')]))
    
    #create a single set
    termSet = frozenset().union(*terms)
    
    #return as a list
    return list(termSet)

In [67]:
row = retest.iloc[0].terms

In [76]:
r = [entry.split(' - ') for entry in row]
from itertools import chain
list(chain(*r))
    

['software architecture',
 'software quality',
 'empirical software engineering',
 'software architecture',
 'software economics',
 'software quality']

# Testing of hdfstorage command line script

In [190]:
dfTotal = pd.read_hdf('DFstore3.h5', 'df')

In [191]:
successFiles = pd.read_hdf('DFstore3.h5', 'files')
failedFiles = pd.read_hdf('DFstore3.h5', 'errors')

In [192]:
for x in successFiles : print x,
print '\n'
for y in failedFiles : print y,

../ArchConfAbstracts/ECSA2007.txt ../ArchConfAbstracts/ECSA2008.txt ../ArchConfAbstracts/ECSA2009.txt ../ArchConfAbstracts/ECSA2010.txt ../ArchConfAbstracts/ECSA2011.txt ../ArchConfAbstracts/ECSA2012.txt ../ArchConfAbstracts/ECSA2013.txt ../ArchConfAbstracts/ECSA2014.txt ../ArchConfAbstracts/QoSA2005.txt ../ArchConfAbstracts/QoSA2006.txt ../ArchConfAbstracts/QoSA2007.txt ../ArchConfAbstracts/QoSA2009.txt ../ArchConfAbstracts/QoSA2010.txt ../ArchConfAbstracts/QoSA2011.txt ../ArchConfAbstracts/QoSA2012.txt ../ArchConfAbstracts/QoSA2013.txt ../ArchConfAbstracts/QoSA2014.txt ../ArchConfAbstracts/WICSA2004.txt ../ArchConfAbstracts/WICSA2005.txt ../ArchConfAbstracts/WICSA2008.txt ../ArchConfAbstracts/WICSA2009.txt ../ArchConfAbstracts/WICSA2011.txt ../ArchConfAbstracts/WICSA2012.txt ../ArchConfAbstracts/WICSA2014.txt 

../ArchConfAbstracts/QoSA2008.txt ../ArchConfAbstracts/WICSA1999.txt ../ArchConfAbstracts/WICSA2001.txt ../ArchConfAbstracts/WICSA2002.txt ../ArchConfAbstracts/WICSA2007.txt


In [196]:
dfTotal.Authors

1         ['Falessi, D', ' Kruchten, P', ' Cantone, G']
2      ['Jerad, C', ' Barkaoui, K', ' Grissa Touzi, A']
3     ['Waignier, G', ' Le Meur, A', 'F', ' Duchien,...
4     ['Falkner, K', ' Balasubramaniam, D', ' Detmol...
5                      ['Brown, A W', ' McDermid, J A']
6     ['Navarro, E', ' Letelier, P', ' Jaen, J', ' R...
7                                        ['Kjaer, K E']
8            ['Santos, A', ' Almeida, F', ' Blanco, V']
9     ['Sant Anna, C', ' Figueiredo, E', ' Garcia, A...
10    ['Lopez Sanz, M', ' Acuna, C J', ' Cuesta, C E...
11       ['Farenhorst, R', ' Lago, P', ' van Vliet, H']
12    ['Losilla, F', ' Vicente Chicote, C', ' Alvare...
13    ['Correia, R', ' Matos, C M P', ' Heckel, R', ...
14                         ['Estevez, E', ' Marcos, M']
15    ['Costa, C', ' Ali, N', ' Perez, J', ' Carsi, ...
16                   ['Cordero, R L', ' Salavert, I R']
17                          ['Gruhn, V', ' Schafer, C']
18                                        ['Garl

# drafts for hdfstorage

In [ ]:
from pandas import HDFStore
store = HDFStore('store.h5')

In [ ]:
store = HDFStore('store.h5')
store['df'] = dfnew
store['files'] = s
store.close()

In [ ]:
files = pd.read_hdf('DFstore.h5', 'files')
dfs = pd.read_hdf('DFstore.h5', 'df')
e = pd.read_hdf('DFstore.h5', 'errors')

In [ ]:
files

In [ ]:
dfs.head()

In [ ]:
s = pd.Series.from_array(['hi', 'bye'])

In [ ]:
e

In [ ]:
files = []
dfs = []
def printFileName(filename):
    print filename

def main():
    args = sys.argv[1:]
    dataDFstore = HDFStore('databaseDF.store')
    
    for filename in args:
        files.append(FileName(filename))
        dfs.append(parser.parseEntrytoDF(FileName(filename)))
        
    dfnew = pd.concat(dfs, axis = 0)
    
    dataDFstore['df'] = dfnew
    
    print files
    dataDFstore['files'] = files
    
    dataDFstore.close()

if __name__ == '__main__':
  main()
#Then from the console, yo